### Retrain the best model from a random search done with classifier_train.py
First load the results from the search and check wich configuration performed best.
Then retrain with some learning rate decay and more epochs.

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pal = sns.color_palette("colorblind")
plt.style.use('plot_style.txt')
from matplotlib.ticker import AutoMinorLocator
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

### Load the data

In [ ]:
df = pd.read_pickle('/mnt/md0/aholmberg/data/raytracing_class_random_25_spherical.pkl')
df.head()

### Find the features in the data

In [ ]:
labels = np.array(df['n_sol'])
sc_pos_d = df['source_pos_d'].to_numpy().astype(np.float32)
sc_pos_phi = df['source_pos_phi'].to_numpy().astype(np.float32)
ant_pos_z = df['antenna_pos_z'].to_numpy().astype(np.float32)
features = np.stack((sc_pos_d, sc_pos_phi, ant_pos_z), axis=1)

### Normalize and split into train and test sets

In [ ]:
features  = features.astype(np.float32)
labels[labels == 2] = 1

norm_x = np.zeros_like(features)
norm_x[:, 0] = features[:, 0] / (np.sqrt(2700**2 + 2000**2))
norm_x[:, 1] = features[:, 1] / (180)
norm_x[:, 2] = features[:, 2] / -(200)

norm_features_train = norm_x[:int(norm_x.shape[0]*0.8)]
norm_features_test = norm_x[int(norm_x.shape[0]*0.8):]
labels_train = labels[:int(labels.shape[0]*0.8)]
labels_test = labels[int(labels.shape[0]*0.8):]

### Load the results from the run and then print the five best and five worst models in latex formated table
putting the to_latex line in a print statement gives a reslut that you can vopy to a latex file or save it in a file directly.

In [ ]:
models = pd.read_pickle(f'class-run-1.pkl')
table = pd.concat([models.sort_values('val_acc', ascending=False).iloc[:5], models.sort_values('val_acc', ascending=False).iloc[-5:]], ignore_index=True, axis=0)
table.to_latex(columns=['lr', 'depth', 'width', 'act', 'val_acc'], )

### Define the classification model 
Should be done by loading the model from the saved models instead of doing it manually like now.

In [ ]:
def get_classifier(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)):
    inputs = layers.Input(shape=(3,))
    x = layers.Dense(48, activation='relu', kernel_initializer='he_normal')(inputs)
    x = layers.Dense(48, activation='relu', kernel_initializer='he_normal')(x)
    x = layers.Dense(48, activation='relu', kernel_initializer='he_normal')(x)
    x = layers.Dense(48, activation='relu', kernel_initializer='he_normal')(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, x)
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()]
        )
    return model

In [ ]:
model = get_classifier()
model.summary()

### Define the callbacks 

In [ ]:
def scheduler(epoch, lr):
    if epoch <= 0:
        return 1e-2
    elif epoch == 1:
        return 1e-3
    else:
        return lr * tf.math.exp(-0.1)


lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler)

checkpoint_filepath = '/mnt/md0/aholmberg/class_models/check/' + 'class_test'
if not os.path.isdir(checkpoint_filepath):
    os.mkdir(checkpoint_filepath)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True
)

### Train with a validation split of 10%

In [ ]:
model.fit(norm_features_train,
          labels_train,
          epochs=20,
          batch_size=512,
          validation_split=0.1,
          callbacks=[model_checkpoint_callback, lr_scheduler],
          shuffle=True)

### Load the best epoch and evaluate

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
y = model.evaluate(norm_features_test, labels_test, batch_size=2048)


### Define the data for confusion matrix

In [ ]:
# total numbers
plot_data = np.array([[y[2], y[5]],
                      [y[4], y[3]]])

# rates
#plot_data = np.array([[y[2]/(y[2] + y[5]), y[5]/(y[5] + y[2])],
#                      [y[4]/(y[4] + y[3]), y[3]/(y[3] + y[4])]])

plot_data = pd.DataFrame(data=plot_data, index=['Solution', 'No solution'], columns=['Solution', 'No solution'])

# rates
#labels = np.array([[f'{y[2]/(y[2] + y[5]):.4f}', f'{y[5]/(y[5] + y[2]):.4f}'],
#                   [f'{y[4]/(y[4] + y[3]):.4f}', f'{y[3]/(y[3] + y[4]):.4f}']])

# total numbers
labels = np.array([[f'{y[2]:.2E}', f'{y[5]:.2E}'],
                   [f'{y[4]:.2E}', f'{y[3]:.2E}']])


### Plot confusion matrix

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5))
sns.heatmap(plot_data, annot=labels, fmt='', cmap='Blues')
ax.set_xlabel('Preticted value')
ax.set_ylabel('True value')
#fig.savefig('thesis/Exjobb-rapport/figures/confusion-matrix.pdf', dpi=300)

### Get the raw prediction of the model and save in a dataframe to use with seaborn

In [ ]:
ys = model.predict(norm_features_test, batch_size=4096)
hist_data = pd.DataFrame(np.stack((ys.squeeze(), labels_test), axis=-1), columns=['Prediction', 'True'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
ax = sns.histplot(hist_data, x='Prediction', hue='True', log_scale=(False,True), legend=False, palette=pal[:2])
ax.legend(labels=['Solution', 'No solution'], frameon=False)
ax.set_xlim(0,1)
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
#fig.savefig('thesis/Exjobb-rapport/figures/class_hist.pdf', dpi=300)

### Plot the model

In [ ]:
keras.utils.plot_model(model, "solution_classifier.png", rankdir='hR', show_shapes=True, show_layer_names=False, dpi=300)